In [13]:
import os
import numpy as np
from scipy import interpolate
import scipy.io as sio
import Rbeast as rb
from SLM_tools import *
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
energy = sio.loadmat(r"C:\Users\User\OneDrive - mail.tau.ac.il\Documents\Final Project\Project2-Omri and Idan\Results\06_30_2022_10_12_02_mu_0__0_2__2_8_Js__4_0_runs_2\2\3\energy_vec_mu_1.6_energy_4.0_run_num_1_total_num_target_2.mat")
energy = energy["foo"]

In [3]:
distance = sio.loadmat(r'C:\Users\User\OneDrive - mail.tau.ac.il\Documents\Final Project\Project2-Omri and Idan\Results\06_30_2022_10_12_02_mu_0__0_2__2_8_Js__4_0_runs_2\2\3\distance_vec_mu_0.0_energy_4.0_run_num_1_total_num_target_2.mat')
distance = distance["foo"]

In [4]:
energy_length = energy.shape[1]
t = np.floor(np.linspace(0, energy_length-1, energy_length // 1000)).astype(int)

In [5]:
energy = energy[:,t]
distance = distance[t,:]

In [6]:
o, cp, mean_trend = SLM_tools.beast(energy)


INFO: To supress printing the parameers in beast(),      set print.options = 0 
INFO: To supress printing the parameers in beast_irreg(),set print.options = 0 
INFO: To supress printing the parameers in beast123(),   set extra.printOptions = 0  
INFO: To supress warning messages in beast(),            set quiet = 1 
INFO: To supress warning messages in beast_irreg(),      set quiet = 1 
INFO: To supress warning messages in beast123(),         set extra.quiet = 1  

#--------------------------------------------------#
#       Brief summary of Input Data                #
#--------------------------------------------------#
Data Dimension: One signal of length 50000
IsOrdered     : Yes, ordered in time
IsRegular     : Yes, evenly spaced at interval of  1 (unknown unit)
hasSeasonCmpnt: False | no periodic or seasonal component. The model Y=Trend+Error is fitted.
HasOutlierCmpt: False | If true, Y=Trend+Outlier+Error (experimental) is fitted instead of Y=Trend+Error 
Detrend       : False 

In [7]:
import pickle
with open("beast_output.pkl", "wb") as f:
    pickle.dump(o,f)

In [14]:
A = SLM_tools.segment_data(energy, distance,mean_trend,cp)

In [15]:
print(A)

[array([-105.52959116,  -73.32456291, -141.10580058, -139.91271916,
        -83.23011829]), array([39.48848768, 23.89291686, 14.12508997, 13.73590672, 18.70533572]), array([ 19.3547404 , -30.45998785,  18.33530571,  23.29280502,
         4.80029862]), array([-0.01426697, -0.00664487, -0.0001718 ,  0.00439264, -0.00803112]), array([ 6066.,  5834., 17481.,  6502.,  5495.]), array([0., 0., 0., 0., 0.]), array([nan, nan, nan, nan, nan])]


In [16]:
from IPython.display import display
display(A)

[array([-105.52959116,  -73.32456291, -141.10580058, -139.91271916,
         -83.23011829]),
 array([39.48848768, 23.89291686, 14.12508997, 13.73590672, 18.70533572]),
 array([ 19.3547404 , -30.45998785,  18.33530571,  23.29280502,
          4.80029862]),
 array([-0.01426697, -0.00664487, -0.0001718 ,  0.00439264, -0.00803112]),
 array([ 6066.,  5834., 17481.,  6502.,  5495.]),
 array([0., 0., 0., 0., 0.]),
 array([nan, nan, nan, nan, nan])]

In [ ]:
print("hello")